# 2) ML Project - Air Pollution - Data Exploration - Merging Additional Databases

<font color='blue'> Scripts by Kevin Jeswani, Junaid Zafar, & Laiba Shah 
<br>
WeCloudData Bootcamp 2022 (Part-time Cohort)<br> </font>

First we take a look at a dataset for air pollutions deaths due over time for different countries. We visualize data and develop the linear regression models on the regional and national level. The linear regression models are valuable at predicting the general trend in deaths due over time and be used to predict future deaths, where past data appears to be highly linear. It has severe limitations for many countries. Polynomial models could be explored. To explore the interaction with other features, we proceed to merge different datasets between death due to environmental factors and socio-demographic/economic factors. We then examine them in 3 tree-based machine learning models.

The set of scripts in the folder 'AirPollution_Scripts' are segmented for the purpose of clarity & convenience <br>
The following is the suggested order for running the scripts:
- 'MLProject_AirPollution_Viz_LR.ipynb' - loading the initial Kaggle air pollution deaths dataset, processing, visualizing the base dataset, linear regression examination and future predictions.
- 'MLProject_DataProcessing_OECD_WB_GBD.ipynb' **This Script** - loading raw data from world bank and GBD/OECD datasets, cleaning, and merging usable data. We test the use of fuzzy data set merging.
- 'MLProject_Exploratory_ML_Models.ipynb' - loading the cleaning & merged dataset, testing out KNN regressor, and model selection with PyCaret, and discussion of models/insights in modelling data.
- 'MLProject_ML_Models_refined.ipynb' - development of selected machine learning models (Extra Trees, Random Forest, XGBoost), cross-validation, & Ensemble methods; discussion of model results

The aim is to explore the intial dataset (https://www.kaggle.com/datasets/akshat0giri/death-due-to-air-pollution-19902017?resource=download), supplement it, infer trends, and make predictions on future deaths due to environmental factors. 
The data is then supplement with:
- Global Burden of Disease Study 2019 Results via OECD.stats: https://stats.oecd.org/index.aspx?lang=en# (where the Kaggle dataset seems to have extracted and cleaned data from)
- World Bank Development Indicators: https://databank.worldbank.org/source/world-development-indicators#
<br>

## 1.0 Pre-processing

### 1.1 Standard Imports

In [78]:
#Data Processing
import numpy as np 
import pandas as pd
import csv

#Plotting
import plotly.express as px
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

#For merging datasets  with slight changes in country names
import difflib
from difflib import SequenceMatcher
import fuzzymatcher
from fuzzymatcher import link_table, fuzzy_left_join

#Machine Learning - Regression
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

## 2.0 Data Examination & Processing

In [2]:
root = r'C:\Users\kevin\Documents\WCD_Bootcamp\ML\ML_Project\Scripts'
root_raw=r'C:\Users\kevin\Documents\WCD_Bootcamp\ML\ML_Project\Scripts\Raw Data'
root_proc=r'C:\Users\kevin\Documents\WCD_Bootcamp\ML\ML_Project\Scripts\Processed'

### 2.1 Load Data into DF & Examine Contents - Air Pollution

In [3]:
fid = '\Mortality_AirPollution.csv'
df_in_AirPollution = pd.read_csv(root_raw+fid)
df_in_AirPollution

,COU,Country,VAR,Variable,RISK,Risk,SEX,Sex,AGE,Age,...,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",PM_2_5_OUT,Ambient Particulate Matter,BOTH,Both,ALL,All,...,1990,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,139.187,NaN,NaN
1,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",PM_2_5_OUT,Ambient Particulate Matter,BOTH,Both,ALL,All,...,1991,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,134.172,NaN,NaN
2,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",PM_2_5_OUT,Ambient Particulate Matter,BOTH,Both,ALL,All,...,1992,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,132.423,NaN,NaN
3,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",PM_2_5_OUT,Ambient Particulate Matter,BOTH,Both,ALL,All,...,1993,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,128.634,NaN,NaN
4,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",PM_2_5_OUT,Ambient Particulate Matter,BOTH,Both,ALL,All,...,1994,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,127.742,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686875,LAC,Latin America and Caribbean,MOR_V,Premature deaths,AIRPOL_IN,Household air pollution from solid fuels,MALE,Male,MORETHAN64,More than 64 years old,...,2015,NBR,Number,0,Units,NaN,NaN,17331.000,NaN,NaN
686876,LAC,Latin America and Caribbean,MOR_V,Premature deaths,AIRPOL_IN,Household air pollution from solid fuels,MALE,Male,MORETHAN64,More than 64 years old,...,2016,NBR,Number,0,Units,NaN,NaN,16900.000,NaN,NaN
686877,LAC,Latin America and Caribbean,MOR_V,Premature deaths,AIRPOL_IN,Household air pollution from solid fuels,MALE,Male,MORETHAN64,More than 64 years old,...,2017,NBR,Number,0,Units,NaN,NaN,16403.000,NaN,NaN
686878,LAC,Latin America and Caribbean,MOR_V,Premature deaths,AIRPOL_IN,Household air pollution from solid fuels,MALE,Male,MORETHAN64,More than 64 years old,...,2018,NBR,Number,0,Units,NaN,NaN,15908.000,NaN,NaN


In [4]:
#Inspect variables provided (measures) - These are the reduced variables from the original OECD dataset
df_in_AirPollution['Variable'].unique()

array(['Premature deaths, per million inhabitants',
       'Premature deaths, % total premature deaths', 'Premature deaths'],
      dtype=object)

In [5]:
#Inspect the Risks in the dataset (premature deaths due to types environmental concerns)
df_in_AirPollution['Risk'].unique()

array(['Ambient Particulate Matter', 'Ambient Ozone',
       'Household air pollution from solid fuels'], dtype=object)

In [6]:
#Inspect the short-hand risk codes - not necessary
df_in_AirPollution['RISK'].unique()

array(['PM_2_5_OUT', 'O3', 'AIRPOL_IN'], dtype=object)

In [7]:
#Flag Codes not useful
df_in_AirPollution['Flag Codes'].isnull().sum()

686880

In [8]:
#Flags not useful
df_in_AirPollution['Flags'].isnull().sum()

686880

In [9]:
#Inspect Reference Period - not useful
df_in_AirPollution['Reference Period'].isnull().sum()

686880

In [10]:
#Inspect Reference Period Code - not useful
df_in_AirPollution['Reference Period Code'].isnull().sum()

686880

In [11]:
#Inspect PowerCode - to describe whether metric is already in the presented units or is to another power ex. millions
df_in_AirPollution[df_in_AirPollution['PowerCode'].notnull()]

,COU,Country,VAR,Variable,RISK,Risk,SEX,Sex,AGE,Age,...,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",PM_2_5_OUT,Ambient Particulate Matter,BOTH,Both,ALL,All,...,1990,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,139.187,NaN,NaN
1,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",PM_2_5_OUT,Ambient Particulate Matter,BOTH,Both,ALL,All,...,1991,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,134.172,NaN,NaN
2,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",PM_2_5_OUT,Ambient Particulate Matter,BOTH,Both,ALL,All,...,1992,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,132.423,NaN,NaN
3,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",PM_2_5_OUT,Ambient Particulate Matter,BOTH,Both,ALL,All,...,1993,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,128.634,NaN,NaN
4,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",PM_2_5_OUT,Ambient Particulate Matter,BOTH,Both,ALL,All,...,1994,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,127.742,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686875,LAC,Latin America and Caribbean,MOR_V,Premature deaths,AIRPOL_IN,Household air pollution from solid fuels,MALE,Male,MORETHAN64,More than 64 years old,...,2015,NBR,Number,0,Units,NaN,NaN,17331.000,NaN,NaN
686876,LAC,Latin America and Caribbean,MOR_V,Premature deaths,AIRPOL_IN,Household air pollution from solid fuels,MALE,Male,MORETHAN64,More than 64 years old,...,2016,NBR,Number,0,Units,NaN,NaN,16900.000,NaN,NaN
686877,LAC,Latin America and Caribbean,MOR_V,Premature deaths,AIRPOL_IN,Household air pollution from solid fuels,MALE,Male,MORETHAN64,More than 64 years old,...,2017,NBR,Number,0,Units,NaN,NaN,16403.000,NaN,NaN
686878,LAC,Latin America and Caribbean,MOR_V,Premature deaths,AIRPOL_IN,Household air pollution from solid fuels,MALE,Male,MORETHAN64,More than 64 years old,...,2018,NBR,Number,0,Units,NaN,NaN,15908.000,NaN,NaN


In [12]:
df_in_AirPollution['PowerCode'].unique()
#in 'Units' or in 'Millions' - not useful already in the 'Risk' descriptor

array(['Units'], dtype=object)

In [13]:
#Inspect PowerCode Code - not useful can just use PowerCode
df_in_AirPollution['PowerCode Code'].unique()

array([0], dtype=int64)

In [14]:
#Inspect missing data points
df_in_AirPollution['Value'].isnull().sum()

0

In [15]:
#Inspect missing data points - potentially 0s replaced null values
df_in_AirPollution[df_in_AirPollution['Value']==0].count()

COU                      94008
Country                  94008
VAR                      94008
Variable                 94008
RISK                     94008
Risk                     94008
SEX                      94008
Sex                      94008
AGE                      94008
Age                      94008
YEA                      94008
Year                     94008
Unit Code                94008
Unit                     94008
PowerCode Code           94008
PowerCode                94008
Reference Period Code        0
Reference Period             0
Value                    94008
Flag Codes                   0
Flags                        0
dtype: int64

In [16]:
df_in_AirPollution['Country'].unique()

array(['Australia', 'Austria', 'Belgium', 'Afghanistan', 'Albania',
       'Algeria', 'American Samoa', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Belarus', 'Benin', 'Bermuda',
       'Burkina Faso', 'Botswana', 'Congo', 'Barbados', 'Brazil',
       'Central African Republic', 'Belize', 'Cameroon', 'Bolivia',
       'Brunei Darussalam', 'Bhutan', 'Bosnia and Herzegovina',
       'Bulgaria', 'Canada', 'Czech Republic', 'Denmark', 'Burundi',
       'Cambodia', 'Cabo Verde', 'Chad', 'Chile',
       "China (People's Republic of)", 'Colombia', 'Comoros',
       'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba',
       "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Egypt', 'El Salvador',
       'Finland', 'France', 'Greece', 'Cyprus', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Ethiopia', 'Fiji', 

There are regional/economic groups: ['European Union (28 countries)', 'OECD - Total',
       'OECD - Europe', 'Middle East and North Africa', 'United States',
       'G7', 'World', 'ASEAN',
       'G20',
       'BRIICS economies - Brazil, Russia, India, Indonesia, China and South Africa',
       'South Sudan ', 'Euro area (19 countries)', 'OECD Asia Oceania',
       'OECD America', 'Eastern Europe, Caucasus and Central Asia',
       'Latin America and Caribbean'
       'European Union (27 countries)'], dtype=objec

In [17]:
#many values reduced to 0, which can be misleading since the variables are given in per million or a percentage,
#where 3 decimal places are drowned out - had a hard time donwloading the data with more decimal places even via API
df_in_AirPollution.loc[(df_in_AirPollution['Value']==0) & (df_in_AirPollution['Risk']=='Ambient Ozone')]['Country'].unique()

array(['American Samoa', 'Andorra', 'Antigua and Barbuda', 'Bahamas',
       'Bermuda', 'Australia', 'Austria', 'Belgium', 'Afghanistan',
       'Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia',
       'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Benin',
       'Botswana', 'Congo', 'Barbados', 'Belize',
       'Bosnia and Herzegovina', 'Central African Republic',
       'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Bolivia',
       'Bhutan', 'Canada', 'Cameroon', 'Brazil', 'Cabo Verde', 'Comoros',
       "Côte d'Ivoire", 'Czech Republic', 'Denmark', 'Burundi',
       'Cambodia', 'Chad', 'Chile', "China (People's Republic of)",
       'Colombia', 'Costa Rica', 'Croatia', 'Cuba',
       "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Egypt', 'El Salvador',
       'Djibouti', 'Dominica', 'Equatorial Guinea', 'Fiji', 'Grenada',
       'Finland', 'France', 'Greece', 'Cyprus', 'Dominican Republic',
       'Ecuador', 'Eritrea', 'Estonia', '

### 2.2 Data Cleaning - Air Pollution

In [18]:
#Reduced_df=
df_airpoll=df_in_AirPollution

In [19]:
#Individual male-female breakdown not necessary
df_airpoll.drop(df_airpoll[(df_airpoll['Sex']=='Male')|(df_airpoll['Sex']=='Female')].index, inplace=True)

In [20]:
#Individual age breakdown not necessary
df_airpoll.drop(df_airpoll[df_airpoll['Age']!='All'].index, inplace=True)

In [21]:
#Drop un-necessary columns - the dataset now contains total population data (no longer broken down by age and sex)
df_airpoll=df_airpoll.drop(['PowerCode','PowerCode Code','Unit Code','Unit','VAR','RISK','SEX',
                 'Reference Period Code','Reference Period','Flag Codes','Flags','YEA','Age','AGE','Sex'],axis=1)

In [22]:
#specify a different dataset for the regions & remove those regional aggregations in the base set
regions=['European Union (28 countries)', 'OECD - Total',
       'OECD - Europe', 'Middle East and North Africa',
       'G7', 'World', 'ASEAN',
       'G20',
       'BRIICS economies - Brazil, Russia, India, Indonesia, China and South Africa',
       'Euro area (19 countries)', 'OECD Asia Oceania',
       'OECD America', 'Eastern Europe, Caucasus and Central Asia',
       'Latin America and Caribbean',
       'European Union (27 countries)']
df_airpoll_regions=df_airpoll[df_airpoll['Country'].isin(regions)]
df_airpoll.drop(df_airpoll[df_airpoll['Country'].isin(regions)].index,inplace=True)

Split the data by variable - best variable to use = 'Premature deaths' (since it has not yet been normalized and the values are less likely to be reverted to a zero)

In [23]:
df_airpoll_tdeaths=df_airpoll[df_airpoll['Variable']=='Premature deaths']
df_airpoll_tdeaths_r=df_airpoll_regions[df_airpoll_regions['Variable']=='Premature deaths']

In [24]:
#store the countries list for merging with other dataset
countries = list(df_airpoll_tdeaths['Country'].unique())

In [25]:
#National Pre-mature Deaths due to Air Pollution Types
AirPollDeaths_c=df_airpoll_tdeaths.pivot_table('Value',['Country','COU','Year'],'Risk')
#Regional Pre-mature Deaths due to Air Pollution Types
AirPollDeaths_r=df_airpoll_tdeaths_r.pivot_table('Value',['Country','Year'],'Risk')

In [26]:
#Write cleaned data to csv
fid='AirPollutionDeaths_Countries.csv'
AirPollDeaths_c.to_csv(root_proc+fid)
fid='AirPollutionDeaths_Regions.csv'
AirPollDeaths_r.to_csv(root_proc+fid)

### 2.3 Load Data into DF & Examine Contents & Clean Data - Water

In [27]:
fid = '\Mortality_Water.csv'
df_in_Water = pd.read_csv(root_raw+fid)
#df_in_Water

In [28]:
df_in_Water.head()

,COU,Country,VAR,Variable,RISK,Risk,SEX,Sex,AGE,Age,...,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",UWATS,Unsafe water source,BOTH,Both,ALL,All,...,1990,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,0.501,NaN,NaN
1,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",UWATS,Unsafe water source,BOTH,Both,ALL,All,...,1991,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,0.556,NaN,NaN
2,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",UWATS,Unsafe water source,BOTH,Both,ALL,All,...,1992,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,0.628,NaN,NaN
3,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",UWATS,Unsafe water source,BOTH,Both,ALL,All,...,1993,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,0.703,NaN,NaN
4,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",UWATS,Unsafe water source,BOTH,Both,ALL,All,...,1994,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,0.765,NaN,NaN


In [29]:
df_waterpoll=df_in_Water

In [30]:
pd.set_option('display.max_rows',10)

In [31]:
df_waterpoll.drop(df_waterpoll[(df_waterpoll['Sex']=='Male')|(df_waterpoll['Sex']=='Female')].index, inplace=True)

In [32]:
#Individual age breakdown not necessary
df_waterpoll.drop(df_waterpoll[df_waterpoll['Age']!='All'].index, inplace=True)

In [33]:
#Drop un-necessary columns - the dataset now contains total population data (no longer broken down by age and sex)
df_waterpoll=df_waterpoll.drop(['PowerCode','PowerCode Code','Unit Code','Unit','VAR','RISK','SEX',
                 'Reference Period Code','Reference Period','Flag Codes','Flags','YEA','Age','AGE','Sex'],axis=1)

In [34]:
#specify a different dataset for the regions & remove those regional aggregations in the base set
df_waterpoll_regions=df_waterpoll[df_waterpoll['Country'].isin(regions)]
df_waterpoll.drop(df_waterpoll[df_waterpoll['Country'].isin(regions)].index,inplace=True)

Split the data by variable - best variable to use = 'Premature deaths' (since it has not yet been normalized and the values are less likely to be reverted to a zero)

In [35]:
#filter for variable = premature deaths only
df_waterpoll_tdeaths=df_waterpoll[df_waterpoll['Variable']=='Premature deaths']
df_waterpoll_tdeaths_r=df_waterpoll_regions[df_waterpoll_regions['Variable']=='Premature deaths']

In [36]:
#National Pre-mature Deaths due to Water Pollution Types
WaterPollDeaths_c=df_waterpoll_tdeaths.pivot_table('Value',['Country','COU','Year'],'Risk')
#Regional Pre-mature Deaths due to Water Pollution Types
WaterPollDeaths_r=df_waterpoll_tdeaths_r.pivot_table('Value',['Country','Year'],'Risk')

In [37]:
#Write cleaned data to csv
fid='\WaterPollutionDeaths_Countries.csv'
WaterPollDeaths_c.to_csv(root_proc+fid)
fid='\WaterPollutionDeaths_Regions.csv'
WaterPollDeaths_r.to_csv(root_proc+fid)

### 2.4 Load Data into DF & Examine Contents & Clean Data - Climate Risk

In [38]:
fid = '\Mortality_ClimateRisk.csv'
df_in_climate = pd.read_csv(root_raw+fid)

In [39]:
df_in_climate.head()

,COU,Country,VAR,Variable,RISK,Risk,SEX,Sex,AGE,Age,...,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",HTEMP,High temperature,BOTH,Both,ALL,All,...,1990,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,56.392,NaN,NaN
1,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",HTEMP,High temperature,BOTH,Both,ALL,All,...,1991,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,54.712,NaN,NaN
2,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",HTEMP,High temperature,BOTH,Both,ALL,All,...,1992,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,50.348,NaN,NaN
3,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",HTEMP,High temperature,BOTH,Both,ALL,All,...,1993,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,49.531,NaN,NaN
4,AUS,Australia,MOR_CAP,"Premature deaths, per million inhabitants",HTEMP,High temperature,BOTH,Both,ALL,All,...,1994,1000000HAB,Per 1 000 000 inhabitants,0,Units,NaN,NaN,55.169,NaN,NaN


In [40]:
df_climate=df_in_climate

In [41]:
#Individual male-female breakdown not necessary
df_climate.drop(df_climate[(df_climate['Sex']=='Male')|(df_climate['Sex']=='Female')].index, inplace=True)

In [42]:
#Individual age breakdown not necessary
df_climate.drop(df_climate[df_climate['Age']!='All'].index, inplace=True)

In [43]:
#Drop un-necessary columns - the dataset now contains total population data (no longer broken down by age and sex)
df_climate=df_climate.drop(['PowerCode','PowerCode Code','Unit Code','Unit','VAR','RISK','SEX',
                 'Reference Period Code','Reference Period','Flag Codes','Flags','YEA','Age','AGE','Sex'],axis=1)

In [44]:
#specify a different dataset for the regions & remove those regional aggregations in the base set
df_climate_regions=df_climate[df_climate['Country'].isin(regions)]
df_climate.drop(df_climate[df_climate['Country'].isin(regions)].index,inplace=True)

Split the data by variable - best variable to use = 'Premature deaths' (since it has not yet been normalized and the values are less likely to be reverted to a zero)

In [45]:
#filter for variable = premature deaths only
df_climate_tdeaths=df_climate[df_climate['Variable']=='Premature deaths']
df_climate_tdeaths_r=df_climate_regions[df_climate_regions['Variable']=='Premature deaths']

In [46]:
#National Pre-mature Deaths due to Water Pollution Types
climateDeaths_c=df_climate_tdeaths.pivot_table('Value',['Country','COU','Year'],'Risk')
#Regional Pre-mature Deaths due to Water Pollution Types
climateDeaths_r=df_climate_tdeaths_r.pivot_table('Value',['Country','Year'],'Risk')

In [47]:
#Write cleaned data to csv\
fid='\climateDeaths_Countries.csv'
climateDeaths_c.to_csv(root_proc+fid)
fid='\climateDeaths_Regions.csv'
climateDeaths_r.to_csv(root_proc+fid)

### 2.5 Merge Environmental Deaths DataFrames

We will ommit lead & radon deaths and deaths due to occupational hazards to focus on macro-environmental factors

In [48]:
# Left Join on 'COU' code - Water Pollution into Airpollution
df_OECD_envdeaths=pd.merge(AirPollDeaths_c,WaterPollDeaths_c,how = 'left',on=['COU','Country','Year'])

In [49]:
# Left Join on 'COU' code - ClimateRisk
df_OECD_envdeaths=pd.merge(df_OECD_envdeaths,climateDeaths_c,how = 'left',on=['COU','Country','Year'])

In [50]:
#Change the column names in preparation to be merged with socio-economic features
df_OECD_envdeaths.rename(columns = {'Ambient Ozone':'Deaths_OZone', 'Ambient Particulate Matter':'Deaths_PM2.5',
                                   'Household air pollution from solid fuels':'Deaths_Indoor',
                                   'No access to handwashing facility':'Deaths_Handwashing',
                                   'Unsafe sanitation':'Deaths_Sanitation',
                                   'Unsafe water source':'Deaths_Water',
                                   'High temperature':'Deaths_HighTemp',
                                   'Low temperature':'Deaths_LowTemp'}, inplace = True)

In [51]:
death_type=['Deaths_OZone','Deaths_PM2.5','Deaths_Indoor','Deaths_Handwashing','Deaths_Sanitation','Deaths_Water',
            'Deaths_HighTemp','Deaths_LowTemp']

In [52]:
pd.set_option('display.max_rows', None)
#Examine where deaths are 0 - In this exerise we cannot predict whether there is missing data or the values are actually 0
#The base data from OECD.stat did not have any sort of code or indicator suggesting NaN value 
df_OECD_envdeaths[(df_OECD_envdeaths.iloc[:,3]==0.0)]

Risk                               Deaths_OZone  Deaths_PM2.5  Deaths_Indoor  \
COU Country                  Year                                              
ASM American Samoa           2001           0.0           6.0            8.0   
                             2002           0.0           6.0            8.0   
                             2003           0.0           6.0            7.0   
                             2004           0.0           6.0            7.0   
                             2005           0.0           6.0            7.0   
                             2006           0.0           6.0            7.0   
                             2007           0.0           6.0            7.0   
                             2008           0.0           6.0            7.0   
                             2009           0.0           6.0            8.0   
                             2010           0.0           6.0            8.0   
                             2011           0.0           6.0            8.0   
                             2012           0.0           6.0            7.0   
                             2013           0.0           7.0            7.0   
                             2014           0.0           7.0            7.0   
                             2015           0.0           7.0            7.0   
                             2016           0.0           7.0            6.0   
                             2017           0.0           8.0            6.0   
                             2018           0.0           8.0            6.0   
                             2019           0.0           8.0            6.0   
AND Andorra                  1990           1.0           9.0            0.0   
                             1991           1.0           9.0            0.0   
                             1992           1.0          10.0            0.0   
                             1993           1.0          10.0            0.0   
                             1994           1.0          10.0            0.0   
                             1995           2.0          10.0            0.0   
                             1996           2.0          11.0            0.0   
                             1997           2.0          11.0            0.0   
                             1998           2.0          12.0            0.0   
                             1999           2.0          12.0            0.0   
                             2000           2.0          12.0            0.0   
                             2001           2.0          13.0            0.0   
                             2002           2.0          13.0            0.0   
                             2003           2.0          13.0            0.0   
                             2004           3.0          13.0            0.0   
                             2005           3.0          12.0            0.0   
                             2006           3.0          13.0            0.0   
                             2007           3.0          13.0            0.0   
                             2008           3.0          13.0            0.0   
                             2009           3.0          13.0            0.0   
                             2010           3.0          13.0            0.0   
                             2011           3.0          12.0            0.0   
                             2012           3.0          12.0            0.0   
                             2013           3.0          12.0            0.0   
                             2014           3.0          12.0            0.0   
                             2015           3.0          12.0            0.0   
                             2016           3.0          11.0            0.0   
                             2017           3.0          10.0            0.0   
                             2018           3.0          10.0            0.0   


In [53]:
df_OECD_envdeaths[(df_OECD_envdeaths.iloc[:,0]==0.0)]
#OZone values are generally very low so it appears that 0 deaths are atributed to this category for small island nations

Risk                                       Deaths_OZone  Deaths_PM2.5  \
COU Country                          Year                               
ASM American Samoa                   1990           0.0           4.0   
                                     1991           0.0           5.0   
                                     1992           0.0           5.0   
                                     1993           0.0           5.0   
                                     1994           0.0           5.0   
                                     1995           0.0           5.0   
                                     1996           0.0           6.0   
                                     1997           0.0           5.0   
                                     2000           0.0           5.0   
                                     2001           0.0           6.0   
                                     2002           0.0           6.0   
                                     2003           0.0           6.0   
                                     2004           0.0           6.0   
                                     2005           0.0           6.0   
                                     2006           0.0           6.0   
                                     2007           0.0           6.0   
                                     2008           0.0           6.0   
                                     2009           0.0           6.0   
                                     2010           0.0           6.0   
                                     2011           0.0           6.0   
                                     2012           0.0           6.0   
                                     2013           0.0           7.0   
                                     2014           0.0           7.0   
                                     2015           0.0           7.0   
                                     2016           0.0           7.0   
                                     2017           0.0           8.0   
                                     2018           0.0           8.0   
                                     2019           0.0           8.0   
ATG Antigua and Barbuda              1990           0.0          23.0   
                                     1991           0.0          24.0   
                                     1992           0.0          24.0   
                                     1993           0.0          26.0   
                                     1994           0.0          26.0   
                                     1995           0.0          27.0   
                                     1996           0.0          25.0   
                                     1997           0.0          25.0   
                                     1998           0.0          25.0   
                                     1999           0.0          25.0   
                                     2000           0.0          24.0   
                                     2001           0.0          24.0   
                                     2002           0.0          23.0   
                                     2003           0.0          23.0   
                                     2004           0.0          22.0   
                                     2005           0.0          22.0   
                                     2006           0.0          23.0   
                                     2007           0.0          25.0   
                                     2008           0.0          25.0   
                                     2009           0.0          24.0   
                                     2010           0.0          25.0   
                                     2011           0.0          25.0   
                                     2012           0.0          26.0   
                                     2013           0.0          27.0   
                                     2014           0.0   

In [54]:
#Niue is the only country with all fields being 0 - drop it as there are also no matches in WB dataset
df_OECD_envdeaths.drop('NIU',level=0,axis=0, inplace = True) #dropping on multi-index level 1

### 2.6 Load Data into DF & Examine Contents & Clean Data - World Bank Data

In [55]:
fid='\WB_GDP_Population_Poverty.csv'
df_in_WB = pd.read_csv(root_raw+fid)

In [56]:
#df_in_WB

In [57]:
df_in_WB.drop(df_in_WB[df_in_WB['Series Name'].isnull()].index, inplace=True)

In [58]:
df_in_WB['Series Name'].unique()

array(['Poverty headcount ratio at $6.85 a day (2017 PPP) (% of population)',
       'Poverty headcount ratio at $3.65 a day (2017 PPP) (% of population)',
       'Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)',
       'GDP per capita (current US$)', 'GDP per capita growth (annual %)',
       'GDP per capita, PPP (constant 2017 international $)',
       'GDP per capita, PPP (current international $)',
       'Population growth (annual %)', 'Population, total'], dtype=object)

In [59]:
df_GDP_PPP=df_in_WB[df_in_WB['Series Name']=='GDP per capita, PPP (constant 2017 international $)'].copy()

In [60]:
#Missing data in original dataset denoted as '..'
df_GDP_PPP[df_GDP_PPP.iloc[:,4:-1]=='..'].count().sum()

1070

In [61]:
df_GDP=df_in_WB[df_in_WB['Series Name']=='GDP per capita (current US$)'].copy()

In [62]:
#Missing data in original dataset denoted as '..'
#GDP per capita, current US$ is most complete
df_GDP[df_GDP.iloc[:,4:-1]=='..'].count().sum()

556

In [63]:
pd.set_option('display.max_columns', None)
#df_GDP[df_GDP.iloc[:,3:(2021-1990)]=='...']

In [64]:
df_pop=df_in_WB[df_in_WB['Series Name']=='Population, total'].copy()

In [65]:
# Much less missing data for population
df_pop[df_pop.iloc[:,4:-1]=='..'].count().sum()

43

In [66]:
# Delete unnecessary series name and identifier - already reduced to one variable
df_pop=df_pop.drop(['Series Code','Series Name'],axis=1)

In [67]:
# Delete unnecessary series name and identifier - already reduced to one variable
df_GDP=df_GDP.drop(['Series Code','Series Name'],axis=1)

#### 2.7 Check Potential Matches in Country Name between WB and OECD dataasets

In [68]:
#pd.set_option('display.max_rows', None)
countries_noMatch_WB=list(df_pop[~df_pop['Country Name'].isin(countries)]['Country Name'])

In [70]:
#pd.set_option('display.max_rows', None)
countries_match_WB=list(df_pop[df_pop['Country Name'].isin(countries)]['Country Name'])

We also tried to use difflib to get close matches between data - however interdependencies between packages was hard to manage and we cannot get this to work all the time as we require changes to package for the other notebooks <br>
    Check '..._3' ipynb<br>
    matches=[]<br>
    count=0<br>
    for c in countries_noMatch_WB:<br>
        mc=difflib.get_close_matches(c, countries,n=1)<br>
        if not mc:<br>
            mc = ['NaN']<br>
            matches.extend(mc)<br>
            match_i.extend(mc)<br>
            matches[count]=match_i<br>
            count=+1<br>
     df_cmatching=pd.DataFrame(matches,countries_noMatch_WB).reset_index()

Next we tried fuzzy matcher. Package interdependencies also made this a challenge to ensure it ran every time. We include it here for completeness

df_countries=pd.DataFrame(countries,columns=["Country"]) <br>
df_WB_NoMatch=pd.DataFrame(countries_noMatch_WB,columns=["Country_WB"])<br>
merge_df = fuzzy_left_join(df_pop, df_countries, ["Country Name"], ["Country"])<br>
merge_df = fuzzy_left_join(df_WB_NoMatch,df_countries, ["Country_WB"], ["Country"])<br>
#merge_df[['best_match_score','__id_left','__id_right','Country Name','Country']]<br>
#merge_df=merge_df.drop(index=merge_df.iloc[43:, :].index.tolist(), axis=0,inplace=True)<br>

In [81]:
#Fuzzy matcher and difflib both have strengths and weaknesses, ultimately had to use both to country match
#and copy results into a final csv manually
# Import country name codex
fid='\WB_OECD_CountryMatches.csv'
df_in_CountryMatch = pd.read_csv(root_raw+fid)

In [82]:
#Replace unidentical country names in WB population dataset with names in OECD dataset
for ind,country in enumerate(df_in_CountryMatch['World Bank Name']):
    df_pop.loc[df_pop['Country Name']==country,"Country Name"]=df_in_CountryMatch.iloc[ind,1]

In [83]:
#Replace unidentical country names in WB GDP dataset with names in OECD dataset
for ind,country in enumerate(df_in_CountryMatch['World Bank Name']):
    df_GDP.loc[df_GDP['Country Name']==country,"Country Name"]=df_in_CountryMatch.iloc[ind,1]

#### 2.7.1 Merge World Bank Population and GDP Datasets with Deaths due to Environmental Factors Dataset

In [84]:
# Reset original table
df_pop=df_pop.reset_index()

In [85]:
# Set the index to be the Country/Country Code combindation
df_pop=df_pop.set_index(['Country Name','Country Code'])

In [86]:
# Drop old index number row
df_pop=df_pop.drop(['index'],axis=1)

In [87]:
#convert column labels as integers instead of strings to merge with OECD data
df_pop.columns=list(range(1990,2022))

In [88]:
# Unstack to get same Country-Year Index combination as environmental factors deaths data frame
df_pop_pivot =df_pop.stack()

In [89]:
# Convert the series into a dataframe
df_pop_pivot=pd.DataFrame(df_pop_pivot)

In [90]:
#Rename the incorrect default column label
df_pop_pivot.rename(columns = {0:'Population'}, inplace = True)

In [91]:
#Relabel the year index
df_pop_pivot.index.names=['Country Name', 'Country Code', 'Year']

In [92]:
# Repeat same steps with GDP data - Reset original table
df_GDP=df_GDP.reset_index()
df_GDP=df_GDP.set_index(['Country Name','Country Code'])
df_GDP=df_GDP.drop(['index'],axis=1)
df_GDP.columns=list(range(1990,2022))
df_GDP_pivot =df_GDP.stack()
df_GDP_pivot=pd.DataFrame(df_GDP_pivot)
df_GDP_pivot.rename(columns = {0:'GDP_pCapita'}, inplace = True)
df_GDP_pivot.index.names=['Country Name', 'Country Code', 'Year']

In [93]:
#Left join onto OECD environmental factor deaths
df_WB=pd.merge(df_GDP_pivot,df_pop_pivot,how='left',on=['Country Name','Country Code','Year'])

In [94]:
FinalMerged=pd.merge(df_OECD_envdeaths.reset_index(),df_WB.reset_index(),how='left',left_on=['COU','Year'],
               right_on=['Country Code','Year'],indicator=True)
#Do not yet set the index again for this as more data will be added

In [95]:
FinalMerged[FinalMerged['_merge']=='left_only']['Country'].unique()
#only Taiwan (Niue as well, but it was dropped above) do not have WB data
# Taiwan but this could still be useful with other datasets

array(['Chinese Taipei'], dtype=object)

In [96]:
FinalMerged=FinalMerged.drop(['_merge'],axis=1)

In [97]:
pd.set_option('display.max_rows', 10)
FinalMerged[FinalMerged['Country']!=FinalMerged['Country Name']]

,COU,Country,Year,Deaths_OZone,Deaths_PM2.5,Deaths_Indoor,Deaths_Handwashing,Deaths_Sanitation,Deaths_Water,Deaths_HighTemp,Deaths_LowTemp,Country Name,Country Code,GDP_pCapita,Population
1140,TWN,Chinese Taipei,1990,181.0,5963.0,3336.0,144.0,51.0,80.0,118.0,2130.0,NaN,NaN,NaN,NaN
1141,TWN,Chinese Taipei,1991,187.0,6045.0,3055.0,136.0,35.0,57.0,150.0,2183.0,NaN,NaN,NaN,NaN
1142,TWN,Chinese Taipei,1992,223.0,6249.0,2832.0,133.0,24.0,40.0,95.0,2425.0,NaN,NaN,NaN,NaN
1143,TWN,Chinese Taipei,1993,241.0,6393.0,2589.0,128.0,18.0,30.0,142.0,2317.0,NaN,NaN,NaN,NaN
1144,TWN,Chinese Taipei,1994,275.0,6657.0,2392.0,126.0,14.0,24.0,123.0,2247.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1165,TWN,Chinese Taipei,2015,619.0,11167.0,535.0,224.0,11.0,40.0,212.0,3459.0,NaN,NaN,NaN,NaN
1166,TWN,Chinese Taipei,2016,545.0,11221.0,551.0,238.0,11.0,41.0,250.0,3647.0,NaN,NaN,NaN,NaN
1167,TWN,Chinese Taipei,2017,524.0,11000.0,554.0,239.0,10.0,39.0,296.0,3822.0,NaN,NaN,NaN,NaN
1168,TWN,Chinese Taipei,2018,560.0,11225.0,533.0,238.0,9.0,37.0,210.0,3861.0,NaN,NaN,NaN,NaN


In [98]:
FinalMerged = FinalMerged.astype({"Year":"int"})

### 2.8 Load Data into DF & Examine Contents & Clean Data - World Bank Income Classification WBIC

In [99]:
# Load World Bank Income Classification per country 
# (Calculated as GNI per capita in US$ with the classification system thresholds updatin every year)
fid='\WB_IncomeClassification_Corrected.csv'
df_WBIC= pd.read_csv(root_raw+fid)

In [100]:
#Values are strings representing Low income (L), Lower middle income (LM), Upper middle income (UM), High income (H)
df_WBIC

,Unnamed: 0,Unnamed: 1,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,AFG,Afghanistan,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L
1,ALB,Albania,LM,LM,LM,L,L,L,LM,L,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,UM,UM,LM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM
2,DZA,Algeria,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,LM,LM,LM
3,ASM,American Samoa,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM
4,AND,Andorra,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,VIR,Virgin Islands (U.S.),H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H,H
214,PSE,West Bank and Gaza,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM
215,YEM,"Yemen, Rep.",LM,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,LM,LM,LM,LM,LM,LM,LM,LM,L,L,L,L,L
216,ZMB,Zambia,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,L,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,L


In [101]:
#Rename the columns then set as keys
df_WBIC.rename(columns = {'Unnamed: 0':'Country Code','Unnamed: 1':'Country Name'}, inplace = True)

In [102]:
#Replace unidentical country names in WB Income Class set with names in OECD dataset
for ind,country in enumerate(df_in_CountryMatch['World Bank Name']):
    df_WBIC.loc[df_WBIC['Country Name']==country,"Country Name"]=df_in_CountryMatch.iloc[ind,1]

In [103]:
years_int=list(range(int(1990),int(2022)))

In [104]:
#Set Country Name and Country Code
df_WBIC=df_WBIC.set_index(['Country Name','Country Code'])
#Convert 'Year' column to integer
df_WBIC.columns=years_int

In [105]:
#Stack to get in the same format as other datasets
df_WBIC_pivot =df_WBIC.stack()
#Create a df
df_WBIC_pivot=pd.DataFrame(df_WBIC_pivot)
# Rename the new stacked column as Income Class
df_WBIC_pivot.rename(columns = {0:'Income_Class'}, inplace = True)

In [106]:
#Missing entries
df_WBIC_pivot[df_WBIC_pivot['Income_Class']=='..'].count()
#Overwrite missing entries - assuming that income class does not decrease, take the first value that appears and project it 
#into the past - manually done in excel in the interest of time

Income_Class    0
dtype: int64

In [107]:
#pd.set_option('display.max_rows', 50)

In [108]:
df_WBIC_pivot.index.names=['Country Name', 'Country Code', 'Year']

In [109]:
FinalMerged=pd.merge(FinalMerged.reset_index(),df_WBIC_pivot.reset_index(),how='left',left_on=['COU','Year'],
               right_on=['Country Code','Year'],indicator=True)
#Do not yet set the index again for this as more data will be added

In [110]:
FinalMerged[FinalMerged['_merge']=='left_only']['Country'].unique()
#All entries merged

array([], dtype=object)

In [111]:
FinalMerged[FinalMerged['Country']!=FinalMerged['Country Name_y']]['Country'].unique()
#DPRK and Sao Tome and Principe are correct matches with slight name differences - Taiwan has no data

array(['Chinese Taipei', "Democratic People's Republic of Korea",
       'Sao Tome and Principe'], dtype=object)

In [112]:
FinalMerged=FinalMerged.drop(['_merge','Country Name_y','Country Name_x',
                             'Country Code_x','Country Code_y'],axis=1)

### 2.9 Load Data into DF & Examine Contents & Clean Data - GBD Socio-Economic Index (SoDI or SDI)

In [113]:
# Load GBD Socio-Economic Index (SoDI) per country 
# (GBD is the original source for the OECD deaths due to environmental factors data)
fid='\GBD_SoDI_Corrected.csv'
df_SoDI= pd.read_csv(root_raw+fid)

In [114]:
df_SoDI.isnull().sum().sum()

0

In [115]:
df_SoDI

,Location,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Global,0.511,0.516,0.521,0.525,0.529,0.534,0.538,0.542,0.547,0.551,0.556,0.561,0.566,0.571,0.576,0.581,0.586,0.591,0.596,0.601,0.607,0.612,0.616,0.621,0.626,0.631,0.635,0.641,0.647,0.651
1,"Central Europe, eastern Europe, and central Asia",0.648,0.654,0.662,0.666,0.669,0.672,0.675,0.678,0.681,0.684,0.687,0.690,0.694,0.698,0.704,0.710,0.714,0.719,0.724,0.728,0.732,0.735,0.738,0.742,0.745,0.748,0.751,0.754,0.758,0.760
2,Central Asia,0.551,0.555,0.557,0.558,0.559,0.559,0.560,0.560,0.561,0.563,0.566,0.569,0.574,0.579,0.585,0.591,0.598,0.605,0.611,0.617,0.622,0.627,0.632,0.637,0.642,0.647,0.651,0.655,0.659,0.663
3,Armenia,0.536,0.541,0.541,0.542,0.544,0.546,0.550,0.554,0.559,0.564,0.570,0.577,0.586,0.596,0.606,0.616,0.626,0.637,0.647,0.652,0.658,0.662,0.666,0.670,0.673,0.676,0.679,0.682,0.686,0.689
4,Azerbaijan,0.576,0.578,0.579,0.578,0.576,0.573,0.569,0.565,0.561,0.559,0.559,0.561,0.564,0.569,0.575,0.583,0.594,0.607,0.619,0.628,0.637,0.645,0.652,0.658,0.664,0.669,0.673,0.677,0.680,0.683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,Nigeria,0.305,0.308,0.312,0.315,0.319,0.324,0.329,0.334,0.339,0.344,0.350,0.356,0.363,0.371,0.381,0.392,0.402,0.412,0.422,0.432,0.442,0.451,0.460,0.469,0.478,0.487,0.495,0.503,0.510,0.515
663,Sao Tome and Principe,0.299,0.302,0.306,0.309,0.313,0.317,0.322,0.327,0.332,0.338,0.344,0.351,0.358,0.365,0.373,0.381,0.390,0.398,0.407,0.416,0.424,0.433,0.443,0.452,0.461,0.470,0.478,0.487,0.495,0.502
664,Senegal,0.227,0.233,0.239,0.245,0.251,0.257,0.262,0.267,0.272,0.277,0.282,0.286,0.290,0.295,0.299,0.304,0.308,0.313,0.318,0.324,0.330,0.336,0.342,0.348,0.354,0.361,0.368,0.375,0.382,0.389
665,Sierra Leone,0.207,0.209,0.210,0.212,0.215,0.218,0.218,0.218,0.219,0.218,0.219,0.221,0.224,0.229,0.234,0.239,0.245,0.252,0.260,0.267,0.275,0.283,0.292,0.304,0.314,0.321,0.328,0.335,0.342,0.347


In [116]:
df_SoDI

,Location,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Global,0.511,0.516,0.521,0.525,0.529,0.534,0.538,0.542,0.547,0.551,0.556,0.561,0.566,0.571,0.576,0.581,0.586,0.591,0.596,0.601,0.607,0.612,0.616,0.621,0.626,0.631,0.635,0.641,0.647,0.651
1,"Central Europe, eastern Europe, and central Asia",0.648,0.654,0.662,0.666,0.669,0.672,0.675,0.678,0.681,0.684,0.687,0.690,0.694,0.698,0.704,0.710,0.714,0.719,0.724,0.728,0.732,0.735,0.738,0.742,0.745,0.748,0.751,0.754,0.758,0.760
2,Central Asia,0.551,0.555,0.557,0.558,0.559,0.559,0.560,0.560,0.561,0.563,0.566,0.569,0.574,0.579,0.585,0.591,0.598,0.605,0.611,0.617,0.622,0.627,0.632,0.637,0.642,0.647,0.651,0.655,0.659,0.663
3,Armenia,0.536,0.541,0.541,0.542,0.544,0.546,0.550,0.554,0.559,0.564,0.570,0.577,0.586,0.596,0.606,0.616,0.626,0.637,0.647,0.652,0.658,0.662,0.666,0.670,0.673,0.676,0.679,0.682,0.686,0.689
4,Azerbaijan,0.576,0.578,0.579,0.578,0.576,0.573,0.569,0.565,0.561,0.559,0.559,0.561,0.564,0.569,0.575,0.583,0.594,0.607,0.619,0.628,0.637,0.645,0.652,0.658,0.664,0.669,0.673,0.677,0.680,0.683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,Nigeria,0.305,0.308,0.312,0.315,0.319,0.324,0.329,0.334,0.339,0.344,0.350,0.356,0.363,0.371,0.381,0.392,0.402,0.412,0.422,0.432,0.442,0.451,0.460,0.469,0.478,0.487,0.495,0.503,0.510,0.515
663,Sao Tome and Principe,0.299,0.302,0.306,0.309,0.313,0.317,0.322,0.327,0.332,0.338,0.344,0.351,0.358,0.365,0.373,0.381,0.390,0.398,0.407,0.416,0.424,0.433,0.443,0.452,0.461,0.470,0.478,0.487,0.495,0.502
664,Senegal,0.227,0.233,0.239,0.245,0.251,0.257,0.262,0.267,0.272,0.277,0.282,0.286,0.290,0.295,0.299,0.304,0.308,0.313,0.318,0.324,0.330,0.336,0.342,0.348,0.354,0.361,0.368,0.375,0.382,0.389
665,Sierra Leone,0.207,0.209,0.210,0.212,0.215,0.218,0.218,0.218,0.219,0.218,0.219,0.221,0.224,0.229,0.234,0.239,0.245,0.252,0.260,0.267,0.275,0.283,0.292,0.304,0.314,0.321,0.328,0.335,0.342,0.347


In [117]:
#Replace unidentical country names with names in OECD dataset if mismatches exist
for ind,country in enumerate(df_in_CountryMatch['World Bank Name']):
    df_SoDI.loc[df_SoDI['Location']==country,"Location"]=df_in_CountryMatch.iloc[ind,1]

In [118]:
years_int=list(range(int(1990),int(2020)))

In [119]:
#Set Country Name and Country Code
df_SoDI=df_SoDI.set_index(['Location'])
#Convert 'Year' column to integer
df_SoDI.columns=years_int

In [120]:
#Stack to get in the same format as other datasets
df_SoDI_pivot =df_SoDI.stack()
#Create a df
df_SoDI_pivot=pd.DataFrame(df_SoDI_pivot)
# Rename year
df_SoDI_pivot.index.names=['Location','Year']
# Rename the new stacked column as Income Class
df_SoDI_pivot.rename(columns = {0:'SoDI'}, inplace = True)

In [121]:
FinalMerged=pd.merge(FinalMerged,df_SoDI_pivot.reset_index(),how='left',left_on=['Country','Year'],
               right_on=['Location','Year'],indicator=True)
#Do not yet set the index again for this as more data will be added

In [122]:
pd.set_option('display.max_rows', 50)
#Check Failed merged - do manual overides in the source file in the interest of time
FinalMerged.loc[(FinalMerged._merge=='left_only') & (FinalMerged.Year==1990),["Country","Location","_merge"]]

,Country,Location,_merge
900,Cabo Verde,NaN,left_only
1140,Chinese Taipei,NaN,left_only


In [123]:
FinalMerged

,index,COU,Country,Year,Deaths_OZone,Deaths_PM2.5,Deaths_Indoor,Deaths_Handwashing,Deaths_Sanitation,Deaths_Water,Deaths_HighTemp,Deaths_LowTemp,GDP_pCapita,Population,Income_Class,Location,SoDI,_merge
0,0,AFG,Afghanistan,1990,387.0,2782.0,34372.0,4825.0,2798.0,3702.0,1085.0,7076.0,..,12412311,L,Afghanistan,0.187,both
1,1,AFG,Afghanistan,1991,376.0,2846.0,35392.0,5127.0,3254.0,4309.0,925.0,7610.0,..,13299016,L,Afghanistan,0.191,both
2,2,AFG,Afghanistan,1992,364.0,3031.0,38065.0,5889.0,4042.0,5356.0,908.0,8255.0,..,14485543,L,Afghanistan,0.195,both
3,3,AFG,Afghanistan,1993,367.0,3256.0,41154.0,7007.0,5392.0,7152.0,1159.0,8430.0,..,15816601,L,Afghanistan,0.196,both
4,4,AFG,Afghanistan,1994,387.0,3401.0,43153.0,7421.0,5418.0,7192.0,1398.0,8659.0,..,17075728,L,Afghanistan,0.194,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6055,5995,ZWE,Zimbabwe,2015,50.0,2785.0,10435.0,4328.0,2879.0,4336.0,389.0,742.0,1445.069702,13814642,L,Zimbabwe,0.452,both
6056,5996,ZWE,Zimbabwe,2016,58.0,2723.0,10365.0,4295.0,2798.0,4244.0,464.0,762.0,1464.588957,14030338,L,Zimbabwe,0.459,both
6057,5997,ZWE,Zimbabwe,2017,70.0,2630.0,10257.0,4251.0,2744.0,4193.0,180.0,891.0,1235.189032,14236599,L,Zimbabwe,0.465,both
6058,5998,ZWE,Zimbabwe,2018,69.0,2600.0,10113.0,4153.0,2608.0,4013.0,282.0,767.0,1254.642265,14438812,LM,Zimbabwe,0.471,both


In [124]:
FinalMerged=FinalMerged.drop(['index','Location','_merge'],axis=1)

In [125]:
#pd.set_option('display.max_rows', 50)

In [126]:
FinalMerged

,COU,Country,Year,Deaths_OZone,Deaths_PM2.5,Deaths_Indoor,Deaths_Handwashing,Deaths_Sanitation,Deaths_Water,Deaths_HighTemp,Deaths_LowTemp,GDP_pCapita,Population,Income_Class,SoDI
0,AFG,Afghanistan,1990,387.0,2782.0,34372.0,4825.0,2798.0,3702.0,1085.0,7076.0,..,12412311,L,0.187
1,AFG,Afghanistan,1991,376.0,2846.0,35392.0,5127.0,3254.0,4309.0,925.0,7610.0,..,13299016,L,0.191
2,AFG,Afghanistan,1992,364.0,3031.0,38065.0,5889.0,4042.0,5356.0,908.0,8255.0,..,14485543,L,0.195
3,AFG,Afghanistan,1993,367.0,3256.0,41154.0,7007.0,5392.0,7152.0,1159.0,8430.0,..,15816601,L,0.196
4,AFG,Afghanistan,1994,387.0,3401.0,43153.0,7421.0,5418.0,7192.0,1398.0,8659.0,..,17075728,L,0.194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6055,ZWE,Zimbabwe,2015,50.0,2785.0,10435.0,4328.0,2879.0,4336.0,389.0,742.0,1445.069702,13814642,L,0.452
6056,ZWE,Zimbabwe,2016,58.0,2723.0,10365.0,4295.0,2798.0,4244.0,464.0,762.0,1464.588957,14030338,L,0.459
6057,ZWE,Zimbabwe,2017,70.0,2630.0,10257.0,4251.0,2744.0,4193.0,180.0,891.0,1235.189032,14236599,L,0.465
6058,ZWE,Zimbabwe,2018,69.0,2600.0,10113.0,4153.0,2608.0,4013.0,282.0,767.0,1254.642265,14438812,LM,0.471


In [127]:
FinalMerged[FinalMerged['Population']=='..']

,COU,Country,Year,Deaths_OZone,Deaths_PM2.5,Deaths_Indoor,Deaths_Handwashing,Deaths_Sanitation,Deaths_Water,Deaths_HighTemp,Deaths_LowTemp,GDP_pCapita,Population,Income_Class,SoDI
1762,ERI,Eritrea,2012,30.0,1022.0,4345.0,2701.0,3227.0,4278.0,622.0,139.0,..,..,L,0.346
1763,ERI,Eritrea,2013,31.0,1057.0,4311.0,2693.0,3217.0,4267.0,662.0,122.0,..,..,L,0.353
1764,ERI,Eritrea,2014,33.0,1091.0,4249.0,2607.0,3068.0,4073.0,613.0,139.0,..,..,L,0.360
1765,ERI,Eritrea,2015,38.0,1127.0,4176.0,2559.0,3023.0,4017.0,774.0,131.0,..,..,L,0.367
1766,ERI,Eritrea,2016,41.0,1182.0,4112.0,2534.0,2977.0,3960.0,736.0,129.0,..,..,L,0.375
1767,ERI,Eritrea,2017,42.0,1241.0,4031.0,2490.0,2914.0,3883.0,757.0,114.0,..,..,L,0.382
1768,ERI,Eritrea,2018,42.0,1306.0,3976.0,2418.0,2782.0,3714.0,713.0,146.0,..,..,L,0.390
1769,ERI,Eritrea,2019,44.0,1380.0,3936.0,2388.0,2708.0,3621.0,733.0,107.0,..,..,L,0.396
2912,KWT,Kuwait,1992,5.0,446.0,6.0,7.0,1.0,3.0,75.0,70.0,..,..,H,0.662
2913,KWT,Kuwait,1993,5.0,461.0,6.0,6.0,1.0,2.0,82.0,72.0,..,..,H,0.667


In [128]:
#Write cleaned data to csv\
fid='\MergedData_EnvDeaths_GDP_Pop_Income_SoDI.csv'
FinalMerged.to_csv(root_proc+fid)